# Tutoriel : utiliser les données ouvertes du Service météorologique du Canada pour l'aide à la décision

Les données du Service météorologique du Canada peuvent aisément être intégrées à des outils d'aide à la décision afin de faciliter la prise de décisions critiques. Ce tutoriel vous montrera comment extraire et traiter les données météorologiques brutes à l'aide du langage de programmation [Python](https://www.python.org/) afin de créer un outil d'aide à la décision basé sur la notion du rapport coût-perte.

In [1]:
from owslib.wms import WebMapService
from datetime import datetime
from datetime import timedelta
import re

# Choix des paramètres
layer = 'REPS.DIAG.24_T8.ERGE25'  # Couche de données désirée
y,x = 43.555, -73.55  # Coordonnées de l'endroit pour lequel on souhaite obtenir des prévisions
timezone = -4 # Fuseau horaire local. Dans cet exemple, le fuseau horaire est UTC-04:00
interval = 12 # Pas de temps entre les prédictions
totaltime = 48 # Nombre d'heures totale de la période de prédiction
#starttime = 'AAAA-MM-JJ HH:00:00' # Si date-heure locale de départ spécifique, utiliser cette variable en enlevant le #

# Détermination de l'heure de départ à partir des premières prédictions disponibles
now = datetime.now().replace(microsecond=0)

if now < now.replace(hour=12, minute=0, second=0):
    tn = [now.replace(hour=0, minute=0, second=0) + timedelta(days = 1)]
else:
    tn = [now.replace(hour=12, minute=0, second=0) + timedelta(days = 1)]

# Si date-heure locale de départ spécifique, utiliser la ligne de code suivante en enlevant le # qui la précède
# tn = [datetime.strptime(starttime, '%Y-%m-%d %H:%M:%S') + timedelta(hours=0-timezone)]

# Calcul de l'heure de chaque prédiction
while tn[len(tn) - 1] != (tn[0] + timedelta(hours=totaltime)):
    tn.append(tn[len(tn) - 1] + timedelta(hours=interval))

# Calcul de l'emprise
minx,miny,maxx,maxy = x-0.25,y-0.25,x+0.25,y+0.25

# Connection au WMS
wms = WebMapService('https://geo.weather.gc.ca/geomet?SERVICE=WMS&REQUEST=GetCapabilities', version='1.3.0')

# Boucle pour extraire les probabilités
info = []
pixel_value = []
i = 0
while i <= len(tn)-1:
    info.append(wms.getfeatureinfo(layers=[layer],
                       srs='EPSG:4326',
                       bbox=(minx, miny, maxx, maxy),
                       size=(100, 100),
                       format='image/jpeg',
                       query_layers=[layer],
                       info_format='text/plain',
                       xy=(50, 50),
                       feature_count=1,
                       time=str(tn[i].isoformat()) + 'Z'
                       ))
    # Extraction de la probabilité que la température soit supérieure à 25°C
    text = info[i].read().decode('utf-8')
    pixel_value.append(float(str(re.findall("value_0\s=\s\W\W*\d*.*\d+", text)).strip('[""]').rpartition("'")[2]))
    print('Pour la date-heure ' + tn[i].strftime("%m/%d/%Y %H:%M:%S") + ', la probabilité que t>25°C est ' + str(pixel_value[i]) + '.')
    i += 1

Pour la date-heure 05/09/2020 00:00:00, la probabilité que t>25°C est 0.0.
Pour la date-heure 05/09/2020 12:00:00, la probabilité que t>25°C est 0.0.
Pour la date-heure 05/10/2020 00:00:00, la probabilité que t>25°C est 0.0.
Pour la date-heure 05/10/2020 12:00:00, la probabilité que t>25°C est 0.0.
Pour la date-heure 05/11/2020 00:00:00, la probabilité que t>25°C est 0.0.


In [2]:
#Données de départ
clients_under25 = 150 # Lorsque la température est inférieure à 25°C, la crèmerie a 150 clients par jour en moyenne
clients_above25 = 300 # Lorsque la température est supérieure à 25°C, la crèmerie a 300 clients par jour en moyenne
clients_served = 200 # Chaque employé peut servir 200 clients/jour
salary = 10 # Les employés sont payés 10,00$/h
profit_per_client = 3.5 # Un client rapporte en moyenne 3,50$
hours_per_day = 8 # Les enployés travaillent de l'ouverture à la fermeture, 8h par jour

# Coûts pour ajouter un deuxième employé pendant une journée
cost = hours_per_day*salary

# Profits perdus si un seul employé est présent une journée où la température est supérieure à 25°C
losses = (clients_above25-clients_served)*profit_per_client

# Rapport coût-perte
cost_losses_ratio = cost/losses

print('Le rapport coût-perte est de ' + str(round(cost/losses, 2)) +'.')


Le rapport coût-perte est de 0.23.


In [3]:
i = 0
while i <= len(tn)-1:
    if pixel_value[i] > cost_losses_ratio:
        print('Pour la date-heure ' + tn[i].strftime("%m/%d/%Y %H:%M:%S") + ", il est recommandé d'avoir 2 employés.")
    else:
        print('Pour la date-heure ' + tn[i].strftime("%m/%d/%Y %H:%M:%S") + ", il est recommandé d'avoir un seul employé.")
    i += 1

Pour la date-heure 05/09/2020 00:00:00, il est recommandé d'avoir un seul employé.
Pour la date-heure 05/09/2020 12:00:00, il est recommandé d'avoir un seul employé.
Pour la date-heure 05/10/2020 00:00:00, il est recommandé d'avoir un seul employé.
Pour la date-heure 05/10/2020 12:00:00, il est recommandé d'avoir un seul employé.
Pour la date-heure 05/11/2020 00:00:00, il est recommandé d'avoir un seul employé.
